# Notebook for analysis of Magnum data by concatenating analysis datasets together
The analysis datasets are stored in the netcdf files created by the script magnumrun_postrun_initial_analysis.py and kept in backup on Freia's funscratch directory

### Imports:

In [1]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import xarray as xr 
import scipy.stats as stat
import sys
import os
import glob
import re
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.core.ivdata as iv
import flopter.core.lputils as lp
import flopter.magnum.database as ut
import flopter.core.fitters as fts

In [2]:
import importlib
importlib.reload(xr)

<module 'xarray' from '/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/__init__.py'>

## Construction of the metadata dataframe
Metadata about the analysed xarray datasets is compiled into a pandas dataframe for simple manipulation

In [6]:
# Create analysed dataset metadata 

path_to_datasets = '/home/jleland/data/external/magnum/'
# path_to_datasets = '/home/jleland/data/externy/magnum/'
# path_to_analysed_datasets = 'analysed_2'
# path_to_analysed_datasets = 'analysed_3'
# path_to_analysed_datasets = 'analysed_4'
# path_to_analysed_datasets = 'test'
# path_to_analysed_datasets = 'phobos_test'
path_to_analysed_datasets = 'analysed_3_downsampled'
os.chdir(path_to_datasets)

In [4]:
# analysed_dataset_fns = glob.glob(path_to_analysed_datasets + '/*.nc')
# analysed_dataset_fns.sort()

# analysed_infos = []

# for i, anal_ds in enumerate(analysed_dataset_fns):
#     match = re.search("\/a{1}([0-9]{3})_([0-9]{3})_([0-9]{19})\.nc", anal_ds)
#     shot_index = int(match.group(1))
#     adc_index = int(match.group(2))
#     shot_timestamp = int(match.group(3))
#     shot_time = ut.human_datetime_str(int(match.group(3)))
    
#     ds = xr.open_dataset(anal_ds)
#     time_len = len(ds['time'])
#     sweep_len = len(ds['sweep'])
    
#     analysed_infos.append({
#         'adc_index':adc_index, 
#         'shot_number':shot_index, 
#         'shot_timestamp':shot_timestamp, 
#         'shot_time':shot_time, 
#         'filename':anal_ds,
#         'time_len': time_len,
#         'sweep_len': sweep_len
#     })

# analysed_infos_df = pd.DataFrame(analysed_infos).set_index('adc_index')

In [ ]:
analysed_infos_df.groupby('time_len').count()

In [6]:
analysed_infos_df.groupby('sweep_len').count().plot()

---
## Saving / loading section
The metadata doesn't need to be reconstructed _every_ time, so we can store it in a .csv file output by pandas. The following 2 methods save and load, use whichever is appropriate

In [9]:
metadata_filename = f'{path_to_analysed_datasets}_xr_metadata.csv'

In [10]:
# Loading the metadata .csv file into a pandas dataframe
analysed_infos_df = pd.read_csv(metadata_filename).set_index('adc_index')

In [8]:
# Saving the metadata.csv file from the constructed pandas dataframe
analysed_infos_df.to_csv(metadata_filename)

End of save/load segment

---

## Index choice section

In [11]:
magnum_probes = lp.MagnumProbes()

In [8]:
indices = analysed_infos_df.where(analysed_infos_df.time_len >= 500).dropna().index.values
indices
print(len(indices), len(analysed_infos_df))

5 490


In [6]:
# list(analysed_infos_df.groupby('time_len'))

In [11]:
# Joe's requested shots
indices = [396, 398, 409]

In [7]:
# Axial Scan
indices = [132,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189]

In [12]:
# 100A, 5slm (He), 0.8T
indices = [357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386]

In [14]:
# 100A, 5slm (He), 1.2T
indices = [245, 246, 247, 248, 249, 250, 251, 252, 254, 255, 256, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271]

In [7]:
# 110A, 5slm (H), 1.2T
indices = [320, 321, 322, 323, 324, 325, 327, 328, 329, 330, 331, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346]

In [6]:
indices = [357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386]\
          + [132,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189]\
          + [320, 321, 322, 323, 324, 325, 327, 328, 329, 330, 331, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346]

In [15]:
analysed_infos_df.loc[indices]

,filename,shot_number,shot_time,shot_timestamp,sweep_len,time_len
adc_index,,,,,,
245,analysed_3_downsampled/a232_245_66986653256874...,232,2019-06-04 14:22:39,6698665325687429120,1499,50
246,analysed_3_downsampled/a233_246_66986659387981...,233,2019-06-04 14:25:02,6698665938798135296,798,50
247,analysed_3_downsampled/a234_247_66986665416438...,234,2019-06-04 14:27:22,6698666541643821056,798,50
248,analysed_3_downsampled/a235_248_66986668924591...,235,2019-06-04 14:28:44,6698666892459114496,799,50
249,analysed_3_downsampled/a236_249_66986671586200...,236,2019-06-04 14:29:46,6698667158620034048,799,50
250,analysed_3_downsampled/a237_250_66986678362845...,237,2019-06-04 14:32:23,6698667836284542976,798,50
251,analysed_3_downsampled/a238_251_66986701348806...,238,2019-06-04 14:41:19,6698670134880661504,799,50
252,analysed_3_downsampled/a239_252_66986706512680...,239,2019-06-04 14:43:19,6698670651268073472,799,50
254,analysed_3_downsampled/a241_254_66986714681334...,241,2019-06-04 14:46:29,6698671468133408768,798,50


In [6]:
# for i in analysed_infos_df.loc[indices].index:
#     row = analysed_infos_df.loc[i]
#     print(row)

## Load adc file metadata

In [16]:
os.chdir('/home/jleland/data/external/magnum/')
# os.chdir('/home/jleland/data/externy/magnum/')
meta_data_ds = xr.open_dataset('all_meta_data.nc')
print(meta_data_ds)

<xarray.Dataset>
Dimensions:                 (shot_number: 523, ts_radial_pos: 46)
Coordinates:
  * shot_number             (shot_number) int32 0 1 2 3 4 ... 519 520 521 522
    ts_number               (shot_number) float64 ...
    ts_timestamp            (shot_number) float64 ...
    ts_time                 (shot_number) datetime64[ns] ...
    adc_index               (shot_number) float64 ...
    adc_time                (shot_number) datetime64[ns] ...
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
Data variables:
    adc_filename            (shot_number) object ...
    ts_density              (shot_number, ts_radial_pos) float64 ...
    ts_temperature          (shot_number, ts_radial_pos) float64 ...
    ts_d_density            (shot_number, ts_radial_pos) float64 ...
    ts_d_temperature        (shot_number, ts_radial_pos) float64 ...
    adc_folder              (shot_number) object ...
    adc_calibration_index   (shot_number) object ...
    adc_4_

# Analysis of shot / adc_file index selection

In [17]:
def preprocess_sel(ds, sweep=slice(0,997)):
    return ds.sel(sweep=sweep)

def preprocess_average(ds):
    ds = ds.reset_index('time', drop=True)
    ds_avg = ds.mean('sweep')
    ds_avg = ds_avg.assign({'d_current': ds.std('sweep')['current']})
    return ds_avg

def preprocess_average_downsample(ds, downsample_to=500):
    ds = ds.reset_index('time', drop=True)
    dsf = int(len(ds.time) / downsample_to)
    ds = ds.sel(time=slice(0, len(ds.time), dsf))
    ds_avg = ds.mean('sweep')
    ds_avg = ds_avg.assign({'d_current': ds.std('sweep')['current']})
    return ds_avg

In [18]:
analysed_metadata_oi = analysed_infos_df.loc[indices]

files_oi = analysed_metadata_oi['filename'].values
shot_numbers = analysed_metadata_oi['shot_number'].values
shot_numbers_da = xr.DataArray(shot_numbers, dims=['shot_number'], name='shot_number')
min_sweep_len = analysed_metadata_oi['sweep_len'].min()
print(min_sweep_len)

795


## Attempt at concatenating datasets manually

In [11]:
# ds_list = [xr.open_dataset(fn) for fn in files_oi]

In [16]:
indices_to_pop = []
for i, ds in enumerate(ds_list):
    if len(ds.time) < 500:
        indices_to_pop.append(i)
        continue
#     ds = preprocess_average_downsample(ds)
#     ds = ds.sel(time=slice())
#     ds = preprocess_average(ds)

In [17]:
print(indices_to_pop)

[]


In [11]:
# combined_ds = xr.concat(ds_list, shot_numbers_da, fill_value=np.nan)

# Analysis of Helium shot @120A, 5slm He, 0.8T

In [13]:
os.chdir('/home/jleland/Data')

FileNotFoundError: [Errno 2] No such file or directory: '/home/jleland/Data'

In [14]:
os.chdir('/home/jleland/data/external/magnum')

In [15]:
helium_meta_ds = meta_data_ds.sel(shot_number=shot_numbers)
helium_meta_ds.max('ts_radial_pos')

<xarray.Dataset>
Dimensions:                 (shot_number: 22)
Coordinates:
  * shot_number             (shot_number) int32 361 362 363 364 ... 387 388 389
    ts_number               (shot_number) float64 ...
    ts_timestamp            (shot_number) float64 ...
    ts_time                 (shot_number) datetime64[ns] ...
    adc_index               (shot_number) float64 ...
    adc_time                (shot_number) datetime64[ns] ...
Data variables:
    adc_filename            (shot_number) object '2019-06-06 09h 49m 51s TT_06699321692420543855.adc' ... '2019-06-06 12h 14m 43s TT_06699359024431898401.adc'
    ts_density              (shot_number) float64 1.037e+20 ... 8.97e+19
    ts_temperature          (shot_number) float64 7.225 8.182 ... 7.034 7.593
    ts_d_density            (shot_number) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    ts_d_temperature        (shot_number) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    adc_folder              (shot_number) object '2019-06-06_Leland/' ... '201

In [16]:
tilt_da = helium_meta_ds[['shot_tilt', 'adc_4_probe', 'adc_5_probe']]
tilt_da['shot_tilt'] = tilt_da.shot_tilt.round()
tilt_da


<xarray.Dataset>
Dimensions:       (shot_number: 22)
Coordinates:
  * shot_number   (shot_number) int32 361 362 363 364 365 ... 386 387 388 389
    ts_time       (shot_number) datetime64[ns] ...
    adc_time      (shot_number) datetime64[ns] ...
    ts_number     (shot_number) float64 ...
    ts_timestamp  (shot_number) float64 ...
    adc_index     (shot_number) float64 ...
Data variables:
    shot_tilt     (shot_number) float64 10.0 9.0 8.0 7.0 6.0 ... 3.0 2.0 1.0 0.0
    adc_4_probe   (shot_number) object 'S' 'S' 'S' 'S' 'S' ... 'R' 'R' 'R' 'R'
    adc_5_probe   (shot_number) object 'L' 'L' 'L' 'L' 'L' ... 'B' 'B' 'B' 'B'

In [19]:
# combined_ds = xr.open_mfdataset(files_oi, concat_dim=shot_numbers_da, preprocess=preprocess_average_downsample, parallel=True, combine='nested')
# combined_ds = xr.open_mfdataset(files_oi, concat_dim=shot_numbers_da, preprocess=preprocess_average, parallel=True, combine='nested')
combined_ds = xr.open_mfdataset(files_oi, concat_dim=shot_numbers_da, preprocess=preprocess_average, parallel=False, combine='nested')
combined_ds = combined_ds.load()
combined_ds

<xarray.Dataset>
Dimensions:      (direction: 2, probe: 4, shot_number: 22, time: 50)
Coordinates:
  * probe        (probe) object 'B' 'L' 'R' 'S'
  * direction    (direction) object 'up' 'down'
  * shot_number  (shot_number) int64 361 362 363 364 365 ... 385 386 387 388 389
Dimensions without coordinates: time
Data variables:
    voltage      (shot_number, probe, direction, time) float64 nan nan ... nan
    current      (shot_number, probe, direction, time) float64 nan nan ... nan
    shot_time    (shot_number, probe, direction, time) float64 nan nan ... nan
    start_time   (shot_number, probe, direction) float64 nan nan ... nan nan
    d_current    (shot_number, probe, direction, time) float64 nan nan ... nan

In [20]:
combined_all_ds = xr.merge([combined_ds, helium_meta_ds.rename({'shot_time':'sweep_time'})])
combined_all_ds

<xarray.Dataset>
Dimensions:                 (direction: 2, probe: 4, shot_number: 22, time: 50, ts_radial_pos: 46)
Coordinates:
  * probe                   (probe) object 'B' 'L' 'R' 'S'
  * direction               (direction) object 'up' 'down'
  * shot_number             (shot_number) int64 361 362 363 364 ... 387 388 389
    ts_number               (shot_number) float64 ...
    ts_timestamp            (shot_number) float64 ...
    ts_time                 (shot_number) datetime64[ns] ...
    adc_index               (shot_number) float64 ...
    adc_time                (shot_number) datetime64[ns] ...
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
Dimensions without coordinates: time
Data variables:
    voltage                 (shot_number, probe, direction, time) float64 nan ... nan
    current                 (shot_number, probe, direction, time) float64 nan ... nan
    shot_time               (shot_number, probe, direction, time) float64 nan ... n

In [21]:
# combined_all_ds.to_netcdf('all_combined_ds.nc')
# combined_all_ds.to_netcdf('joe_combined_ds.nc')

In [22]:
anglescan_ds.compute().load()

NameError: name 'anglescan_ds' is not defined

In [23]:
anglescan_ds = xr.merge([combined_ds, helium_meta_ds.rename({'shot_time':'sweep_time'})]).assign_coords(tilt=helium_meta_ds['shot_tilt'].round())
anglescan_ds = anglescan_ds.swap_dims({'shot_number':'tilt'}) #.mean('direction')

# Reorganise to make tilt and probe dimensions
probes_1 = anglescan_ds.sel(probe=['S', 'L']) #.where(np.isfinite(anglescan_ds.voltage), drop=True) 
probes_2 = anglescan_ds.sel(probe=['R', 'B']) #.where(np.isfinite(anglescan_ds.voltage), drop=True)

anglescan_ds = xr.concat([probes_1, probes_2], dim='probe').mean('direction')
anglescan_ds

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


<xarray.Dataset>
Dimensions:                 (probe: 4, tilt: 22, time: 50, ts_radial_pos: 46)
Coordinates:
    adc_time                (tilt) datetime64[ns] 2019-06-06T08:49:41 ... 2019-06-06T11:14:33
    ts_number               (tilt) float64 471.0 472.0 473.0 ... 496.0 497.0
    shot_number             (tilt) int32 361 362 363 364 365 ... 386 387 388 389
    ts_time                 (tilt) datetime64[ns] 2019-06-06T08:49:41.690607 ... 2019-06-06T11:14:33.726839
    ts_timestamp            (tilt) float64 6.699e+18 6.699e+18 ... 6.699e+18
    adc_index               (tilt) float64 357.0 358.0 359.0 ... 385.0 386.0
  * tilt                    (tilt) float64 10.0 9.0 8.0 7.0 ... 3.0 2.0 1.0 0.0
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
  * probe                   (probe) object 'S' 'L' 'R' 'B'
Dimensions without coordinates: time
Data variables:
    voltage                 (tilt, probe, time) float64 -33.19 -33.71 ... -8.424
    current             

In [18]:
colourmap = plt.get_cmap('nipy_spectral')
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=[colourmap(k) for k in np.linspace(0, 1, 12)])

In [38]:
iv_fig, iv_ax = plt.subplots(2, 2, sharex=True)
for i, probe in enumerate(anglescan_ds['probe'].values):
    iv_ax[i%2][i//2].set_title(probe)
    
    ds = anglescan_ds.sel(probe=probe)
    ds.set_coords('voltage')['current'].plot.line(ax=iv_ax[i%2][i//2], x='voltage', hue='tilt')
    iv_ax[i%2][i//2].set_ylabel('I (A)')
    iv_ax[i%2][i//2].set_xlabel(r'$V_P$ (V)')
    
# plt.tight_layout()
iv_fig.suptitle('IV characteristics for He shot @ 0.8T')
plt.show()

In [29]:
iv_fig, iv_ax = plt.subplots(2, 2, sharex=True)
for i, probe in enumerate(anglescan_ds['probe'].values):
    iv_ax[i%2][i//2].set_title(probe)
    
    ds = anglescan_ds.sel(probe=probe)
    ds.set_coords('voltage')['voltage'].plot.line(ax=iv_ax[i%2][i//2], x='time', hue='tilt')
    iv_ax[i%2][i//2].set_ylabel(r'$V_P$ (V)')
    iv_ax[i%2][i//2].set_xlabel('t (ms)')
    
# plt.tight_layout()
plt.show()

In [39]:
# ts_temp = anglescan_ds.ts_temperature.mean(['probe', 'tilt', 'time'])
# ts_d_temp = anglescan_ds.ts_temperature.std(['probe', 'tilt', 'time'])
# ts_dens = anglescan_ds.ts_density.mean(['probe', 'tilt', 'time'])
# ts_d_dens = anglescan_ds.ts_density.std(['probe', 'tilt', 'time'])
ts_temp = anglescan_ds.ts_temperature.mean(['probe', 'tilt'])
ts_d_temp = anglescan_ds.ts_temperature.std(['probe', 'tilt'])
ts_dens = anglescan_ds.ts_density.mean(['probe', 'tilt'])
ts_d_dens = anglescan_ds.ts_density.std(['probe', 'tilt'])

print(ts_temp.shape)

fig, ax = plt.subplots(2)
ax[0].set_title('Temperature')
ax[1].set_title('Density')
# ax[0].errorbar(helium_anglescan_ds.ts_radial_pos, ts_temp, yerr=helium_anglescan_ds.ts_d_temperature.mean('shot_number'), color='silver', ecolor='silver')
ax[0].errorbar(anglescan_ds.ts_radial_pos, ts_temp, yerr=ts_d_temp)
ax[1].errorbar(anglescan_ds.ts_radial_pos, ts_dens, yerr=ts_d_dens)
# ts_temp.plot.line(ax=ax[0])
# ts_dens.plot.line(ax=ax[1])
for i in [0, 1]:
    for probe_pos in [-6, 4, 14, 24]:
        ax[i].axvline(x=probe_pos, color='black', linewidth=1, linestyle='dashed')
plt.show()
fig.suptitle('Thomson Scattering profiles for He shot @ 0.8T')


/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:140: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


(46,)


Text(0.5, 0.98, 'Thomson Scattering profiles for He shot @ 0.8T')

In [31]:
# helium_anglescan_ds.where(helium_anglescan_ds.ts_temperature == helium_anglescan_ds.max('ts_radial_pos')['ts_temperature'])
middle_temps = anglescan_ds.isel(ts_radial_pos=anglescan_ds['ts_temperature'].argmax('ts_radial_pos'))['ts_radial_pos']
middle_dens = anglescan_ds.isel(ts_radial_pos=anglescan_ds['ts_density'].argmax('ts_radial_pos'))['ts_radial_pos']
middle_combo = np.concatenate([middle_temps.values, middle_dens.values])

print(middle_dens.mean().values, middle_dens.std().values / 22)
print(middle_temps.mean().values, middle_temps.std().values / 22)
print(middle_combo.mean(), middle_combo.std() / 22)

1.0190559530800039 0.03739069064835448
-0.8821678229353644 0.04090105998671816
0.06844406507231972 0.058331402026303686


In [32]:
probe_temps = {}
probe_denss = {}
for probe, probe_pos in {'L':6, 'S':-4, 'B':-14, 'R':-24}.items():
    probe_temps[probe] = ts_temp.interp(ts_radial_pos=probe_pos).values
    probe_denss[probe] = ts_dens.interp(ts_radial_pos=probe_pos).values
    
print(probe_temps)
print(probe_denss)

{'L': array(3.58860903), 'S': array(6.04210037), 'B': array(1.92278266), 'R': array(0.40818145)}
{'L': array(5.28222965e+19), 'S': array(8.34363829e+19), 'B': array(1.93914071e+19), 'R': array(2.26544465e+18)}


In [33]:
anglescan_ds

<xarray.Dataset>
Dimensions:                 (probe: 4, tilt: 22, time: 50, ts_radial_pos: 46)
Coordinates:
    adc_time                (tilt) datetime64[ns] 2019-06-06T08:49:41 ... 2019-06-06T11:14:33
    ts_number               (tilt) float64 471.0 472.0 473.0 ... 496.0 497.0
    shot_number             (tilt) int32 361 362 363 364 365 ... 386 387 388 389
    ts_time                 (tilt) datetime64[ns] 2019-06-06T08:49:41.690607 ... 2019-06-06T11:14:33.726839
    ts_timestamp            (tilt) float64 6.699e+18 6.699e+18 ... 6.699e+18
    adc_index               (tilt) float64 357.0 358.0 359.0 ... 385.0 386.0
  * tilt                    (tilt) float64 10.0 9.0 8.0 7.0 ... 3.0 2.0 1.0 0.0
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
  * probe                   (probe) object 'S' 'L' 'R' 'B'
Dimensions without coordinates: time
Data variables:
    voltage                 (tilt, probe, time) float64 -33.19 -33.71 ... -8.424
    current             

In [40]:
# fit and plot for a single probe
probes = ['S', 'L', 'B', 'R']
dens = {label: [] for label in probes}
temps = {label: [] for label in probes}
fig, axes = plt.subplots(3)

probe = 'S'
for tilt in tilt_da['shot_tilt'].values:
#     ds = anglescan_ds.sel(probe=probe, direction='up').max('ts_radial_pos')
    ds = anglescan_ds.sel(probe=probe).max('ts_radial_pos')
    ds = ds.where(np.logical_and(np.isfinite(ds.voltage), ds.tilt == tilt), drop=True)
#     print(ds['current'].values[0])

#     print(shot_iv.keys(), shot_iv['sigma'])
    try:
        shot_iv = iv.IVData(ds['voltage'].values[0],
                    -ds['current'].values[0],
                    ds['shot_time'].values[0],
                    estimate_error_fl=True)
#         iv_fitter = fts.FullIVFitter()
#         shot_fit = iv_fitter.fit_iv_data(shot_iv, sigma=shot_iv['sigma'])
        shot_fit = shot_iv.multi_fit(sat_region=-60)
        dens[probe].append(magnum_probes[probe].get_density(shot_fit.get_isat(), shot_fit.get_temp(), alpha=np.radians(tilt)))
        temps[probe].append(shot_fit.get_temp())
        axes[2].plot(*shot_fit.get_fit_plottables(), label=f'tilt = {tilt}')
        axes[2].plot(*shot_fit.get_raw_plottables(), color='silver')
    except:
        dens[probe].append(np.nan)
        temps[probe].append(np.nan)

axes[0].plot(tilt_da['shot_tilt'].values, dens[probe], 'x', label=probe)
axes[0].axhline(y=probe_denss[probe], color='red', linewidth=0.5, linestyle='dashed')

axes[1].plot(tilt_da['shot_tilt'].values, temps[probe], 'x', label=probe)
axes[1].axhline(y=probe_temps[probe], color='red', linewidth=0.5, linestyle='dashed')

axes[2].legend()

fig.suptitle('Anglescan fitting for He shot @ 0.8T')


/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:223: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:223: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:223: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:223: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:223: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)

Text(0.5, 0.98, 'Anglescan fitting for He shot @ 0.8T')

In [36]:
# fit and plot for a single probe
probes = ['S', 'L', 'B', 'R']
denss = {label: [] for label in probes}
temps = {label: [] for label in probes}

for probe in probes:

    fig, axes = plt.subplots(3)

    for tilt in tilt_da['shot_tilt'].values:
    #     ds = anglescan_ds.sel(probe=probe, direction='up').max('ts_radial_pos')
        ds = anglescan_ds.sel(probe=probe).max('ts_radial_pos')
        ds = ds.where(np.logical_and(np.isfinite(ds.voltage), ds.tilt == tilt), drop=True)
    #     print(ds['current'].values[0])

    #     print(shot_iv.keys(), shot_iv['sigma'])
        try:
            shot_iv = iv.IVData(ds['voltage'].values[0],
                        -ds['current'].values[0],
                        ds['shot_time'].values[0],
                        estimate_error_fl=True)
    #         iv_fitter = fts.FullIVFitter()
    #         shot_fit = iv_fitter.fit_iv_data(shot_iv, sigma=shot_iv['sigma'])
            shot_fit = shot_iv.multi_fit(sat_region=-60)
            temp = shot_fit.get_temp()
            dens = magnum_probes[probe].get_density(shot_fit.get_isat(), shot_fit.get_temp(), alpha=np.radians(tilt))
            
            denss[probe].append(dens)
            temps[probe].append(temp)
            print(len(denss[probe]), len(temps[probe]))
            print(denss[probe], temps[probe])
            axes[2].plot(*shot_fit.get_fit_plottables(), label=f'tilt = {tilt}')
            axes[2].plot(*shot_fit.get_raw_plottables(), color='silver', label=f'{tilt}-fit = {temp:.2g}eV, {dens:.2g}m-3')
        except Exception as e:
            print(e)
#             denss[probe].append(np.nan)
#             temps[probe].append(np.nan)

    axes[0].plot(tilt_da['shot_tilt'].values, denss[probe], 'x', label=probe)
    axes[0].axhline(y=probe_denss[probe], color='red', linewidth=0.5, linestyle='dashed')

    axes[1].plot(tilt_da['shot_tilt'].values, temps[probe], 'x', label=probe)
    axes[1].axhline(y=probe_temps[probe], color='red', linewidth=0.5, linestyle='dashed')

    axes[2].legend()

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:223: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:223: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:223: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:223: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)


1 1
[array([3.59096595e+19])] [7.295757449468269]
unsupported format string passed to numpy.ndarray.__format__
2 2
[array([3.59096595e+19]), array([3.95527331e+19])] [7.295757449468269, 7.169469182710168]
unsupported format string passed to numpy.ndarray.__format__
3 3
[array([3.59096595e+19]), array([3.95527331e+19]), array([4.5837848e+19])] [7.295757449468269, 7.169469182710168, 6.660347565273196]
unsupported format string passed to numpy.ndarray.__format__
4 4
[array([3.59096595e+19]), array([3.95527331e+19]), array([4.5837848e+19]), array([5.09459755e+19])] [7.295757449468269, 7.169469182710168, 6.660347565273196, 6.063647612659211]
unsupported format string passed to numpy.ndarray.__format__


/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:223: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:223: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:223: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:223: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)


5 5
[array([3.59096595e+19]), array([3.95527331e+19]), array([4.5837848e+19]), array([5.09459755e+19]), array([5.22501799e+19])] [7.295757449468269, 7.169469182710168, 6.660347565273196, 6.063647612659211, 5.807978504414076]
unsupported format string passed to numpy.ndarray.__format__
6 6
[array([3.59096595e+19]), array([3.95527331e+19]), array([4.5837848e+19]), array([5.09459755e+19]), array([5.22501799e+19]), array([5.18872175e+19])] [7.295757449468269, 7.169469182710168, 6.660347565273196, 6.063647612659211, 5.807978504414076, 5.575653172340178]
unsupported format string passed to numpy.ndarray.__format__
7 7
[array([3.59096595e+19]), array([3.95527331e+19]), array([4.5837848e+19]), array([5.09459755e+19]), array([5.22501799e+19]), array([5.18872175e+19]), array([4.78534321e+19])] [7.295757449468269, 7.169469182710168, 6.660347565273196, 6.063647612659211, 5.807978504414076, 5.575653172340178, 5.662628706755108]
unsupported format string passed to numpy.ndarray.__format__
8 8
[array

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:223: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:223: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:223: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:223: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)


...

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:223: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:223: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:223: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)


15 15
[array([-3.15637521e+15]), array([-2.12987426e+17]), array([-2.70545146e+17]), array([-3.07818308e+17]), array([-2.78508423e+17]), array([-2.51806035e+17]), array([-1.64449332e+17]), array([-5.47273887e+16]), array([-3.15637521e+15]), array([-2.12987426e+17]), array([-2.70545146e+17]), array([-3.07818308e+17]), array([-2.78508423e+17]), array([-2.51806035e+17]), array([-1.64449332e+17])] [27.722365699622607, 4.880880050323226, 6.177360367534764, 6.103527237069769, 6.794898793876871, 5.876260965302189, 4.196408843999014, 1.706810658236237, 27.722365699622607, 4.880880050323226, 6.177360367534764, 6.103527237069769, 6.794898793876871, 5.876260965302189, 4.196408843999014]
unsupported format string passed to numpy.ndarray.__format__
16 16
[array([-3.15637521e+15]), array([-2.12987426e+17]), array([-2.70545146e+17]), array([-3.07818308e+17]), array([-2.78508423e+17]), array([-2.51806035e+17]), array([-1.64449332e+17]), array([-5.47273887e+16]), array([-3.15637521e+15]), array([-2.129

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:223: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:223: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)


Optimal parameters not found: The maximum number of function evaluations is exceeded.


ValueError: x and y must have same first dimension, but have shapes (22,) and (16, 1)

In [41]:
# fit and plot for all probes
probes = ['S', 'L', 'B', 'R']
denss = {label: [] for label in probes}
temps = {label: [] for label in probes}
fits = {label: [] for label in probes}

fig, axes = plt.subplots(4, 2)
colours = ['blue', 'orange', 'green', 'red']

for i, probe in enumerate(probes):
    for tilt in tilt_da['shot_tilt'].values:
#         ds = helium_anglescan_ds.sel(probe=probe).mean('direction').max('ts_radial_pos')
        ds = anglescan_ds.sel(probe=probe).max('ts_radial_pos')
        ds = ds.where(np.logical_and(np.isfinite(ds.voltage), ds.tilt == tilt), drop=True)
        
        try:
            shot_iv = iv.IVData(ds['voltage'].values[0],
                            -ds['current'].values[0],
                            ds['shot_time'].values[0],
                            estimate_error_fl=True)
#             iv_fitter = fts.FullIVFitter()
#             shot_fit = iv_fitter.fit_iv_data(shot_iv, sigma=shot_iv['sigma'])
            shot_fit = shot_iv.multi_fit(sat_region=-60)
            fits[probe].append(shot_fit)
            denss[probe].append(magnum_probes[probe].get_density(shot_fit.get_isat(), shot_fit.get_temp(), alpha=np.radians(tilt)))
            temps[probe].append(shot_fit.get_temp())
        except :
            denss[probe].append(np.nan)
            temps[probe].append(np.nan)
    
    axes[i][0].set_title(r'$n_e$')
    axes[i][0].set_xlabel(r'$\theta_{\perp}$ / degrees')
    axes[i][0].set_ylabel(r'$n_e$ / m$^{-3}$')
    axes[i][0].plot(tilt_da['shot_tilt'].values, denss[probe], '+', label=probe, color=colours[i])
    axes[i][0].axhline(y=probe_denss[probe], color=colours[i], linewidth=0.5, linestyle='dashed')
    axes[i][0].legend()

    axes[i][1].set_title(r'$T_e$')
    axes[i][1].set_xlabel(r'$\theta_{\perp}$ / degrees')
    axes[i][1].set_ylabel(r'$T_e$ / eV')
    axes[i][1].plot(tilt_da['shot_tilt'].values, temps[probe], '+', label=probe, color=colours[i])
    axes[i][1].axhline(y=probe_temps[probe], color=colours[i], linewidth=0.5, linestyle='dashed')
    axes[i][1].legend()

# plt.tight_layout()
fig.suptitle('Anglescan fitting for all probes on He shot @ 0.8T')

plt.show()

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:223: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:223: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:223: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:223: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:223: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)

In [34]:
fig, axes = plt.subplots(2, 2)
base_fmt_str = r'$\theta$ = {:.2g}, $T_e$={:.2g}, $n_e$={:.2g}'

for i, probe in enumerate(probes[0:3]):
    probe_axes = axes[i//2][i%2]
    for j, fit in enumerate(fits[probe]):
        probe_axes.plot(*fit.get_raw_plottables(), color='silver')
        label_str = base_fmt_str.format(tilt_da.values[j], temps[probe][j], denss[probe][j])
        probe_axes.plot(*fit.get_fit_plottables())
    
    anal_voltage = np.linspace(-100, -20, 100)
    
    for tilt in [1.0,10.0]:
        v_f = -(1/tilt) * 10 * probe_temps[probe]
        anal_current = magnum_probes[probe].get_analytical_iv(anal_voltage, v_f, np.radians(tilt), probe_temps[probe], probe_denss[probe], print_fl=True)
#         print(v_f, tilt, probe_temps[probe], probe_denss[probe])
#         print(anal_current)
        label_str = base_fmt_str.format(tilt, probe_temps[probe], probe_denss[probe])
        probe_axes.plot(anal_voltage, anal_current, label='TS IV ({})'.format(label_str), color='black', linestyle='dashed')
    
    probe_axes.legend()
    probe_axes.set_title(f'Probe {probe}')
    probe_axes.set_ylim(bottom=-0.1)
plt.show()

TypeError: unsupported format string passed to numpy.ndarray.__format__

In [35]:
tilt = 10.0
i = 10
probe = 'S'
print(i, tilt)


positions = np.array([-7, 6])

fig, axes = plt.subplots()
ts_temp.plot()

# axes.axvline(x=positions[0], color='red')
# axes.vlines([positions[0] + (i*10) for i in np.arange(-1, 3)], -0.5, 7.5, color='red')

# axes.axvline(x=positions[1], color='blue')
# axes.vlines([positions[1] + (i*10) for i in np.arange(-1, 3)], -0.5, 7.5, color='blue')

positions = np.linspace(-8, -6, 10)
fig, axes = plt.subplots()

fit = fits[probe][i]
axes.plot(*fit.get_raw_plottables(), color='silver')
label_str = base_fmt_str.format(tilt_da.values[j], temps[probe][i], denss[probe][i])
axes.plot(*fit.get_fit_plottables())

for pos in positions:
    temp = ts_temp.interp(ts_radial_pos=pos).values
    dens = ts_dens.interp(ts_radial_pos=pos).values
    v_f = temp * -10
#     anal_voltage = np.linspace(-100, -20, 100)
    anal_voltage = fit.raw_x
    anal_current = magnum_probes[probe].get_analytical_iv(anal_voltage, v_f, np.radians(tilt), temp, dens, print_fl=True)
    label_str = base_fmt_str.format(tilt, temp, dens)
    axes.plot(anal_voltage, anal_current, label=f'pos = {pos}', linestyle='dashed')

axes.legend()
axes.set_ylim(bottom=0.0)

10 10.0


IndexError: list index out of range

# Non-sweep Averaging

In [36]:
combined_ds = xr.open_mfdataset(files_oi, concat_dim=shot_numbers_da, preprocess=preprocess_sel)
combined_ds = combined_ds.load()
combined_ds

<xarray.Dataset>
Dimensions:      (direction: 2, probe: 4, shot_number: 22, sweep: 997, time: 150)
Coordinates:
  * probe        (probe) object 'B' 'L' 'R' 'S'
  * time         (time) float64 0.0 0.0001 0.0001 ... 0.0049 0.0049 0.0049
  * direction    (direction) object 'up' 'down'
  * shot_number  (shot_number) int64 329 330 331 332 333 ... 349 350 351 352 353
Dimensions without coordinates: sweep
Data variables:
    voltage      (shot_number, probe, direction, sweep, time) float64 nan ... nan
    current      (shot_number, probe, direction, sweep, time) float64 nan ... nan
    shot_time    (shot_number, probe, direction, sweep, time) float64 nan ... nan
    start_time   (shot_number, probe, direction, sweep) float64 nan nan ... nan

In [37]:
hydrogen_anglescan_ds = xr.merge([combined_ds, helium_meta_ds.rename({'shot_time':'sweep_time'})]).assign_coords(tilt=helium_meta_ds['shot_tilt'].round())
hydrogen_anglescan_ds

<xarray.Dataset>
Dimensions:                 (direction: 2, probe: 4, shot_number: 22, sweep: 997, time: 150, ts_radial_pos: 46)
Coordinates:
  * probe                   (probe) object 'B' 'L' 'R' 'S'
  * time                    (time) float64 0.0 0.0001 0.0001 ... 0.0049 0.0049
  * direction               (direction) object 'up' 'down'
  * shot_number             (shot_number) int32 329 330 331 332 ... 351 352 353
    ts_number               (shot_number) float64 ...
    ts_timestamp            (shot_number) float64 ...
    ts_time                 (shot_number) datetime64[ns] ...
    adc_index               (shot_number) float64 ...
    adc_time                (shot_number) datetime64[ns] ...
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
    tilt                    (shot_number) float64 1.0 2.0 3.0 ... 8.0 9.0 10.0
Dimensions without coordinates: sweep
Data variables:
    voltage                 (shot_number, probe, direction, sweep, time) float64 na

In [38]:
probe = 'R'

# shot_number = 343
shot_number = 345

In [39]:
weird_ds = hydrogen_anglescan_ds.sel(probe=probe, shot_number=shot_number).max('ts_radial_pos')
# print(weird_ds)
# weird_ds = weird_ds.where(np.isfinite(weird_ds.voltage), drop=True)
print(weird_ds)

<xarray.Dataset>
Dimensions:                 (direction: 2, sweep: 997, time: 150)
Coordinates:
    probe                   <U1 'R'
  * time                    (time) float64 0.0 0.0001 0.0001 ... 0.0049 0.0049
  * direction               (direction) object 'up' 'down'
    shot_number             int32 345
    ts_number               float64 ...
    ts_timestamp            float64 ...
    ts_time                 datetime64[ns] ...
    adc_index               float64 ...
    adc_time                datetime64[ns] ...
    tilt                    float64 2.0
Dimensions without coordinates: sweep
Data variables:
    voltage                 (direction, sweep, time) float64 2.365 nan ... nan
    current                 (direction, sweep, time) float64 0.0001562 ... nan
    shot_time               (direction, sweep, time) float64 1.015 nan ... nan
    start_time              (direction, sweep) float64 1.015 1.025 ... 10.98
    adc_filename            object '2019-06-05 16h 47m 40s TT_06699058

/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:220: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwds)


In [45]:
plt.figure()
for sweep in weird_ds['sweep'].values[::10]:
    sweep_ds = weird_ds.sel(sweep=sweep)
#     plt.plot(sweep_ds['time'].values, sweep_ds['voltage'].values)
    plt.plot(sweep_ds['voltage'].values, sweep_ds['current'].values)
    
sweep_avg = weird_ds.mean('sweep')
plt.plot(sweep_avg['voltage'].values, sweep_avg['current'].values, 'k', label='Average')
plt.show()

/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:159: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [44]:
zeroed_ds = weird_ds - sweep_avg
bins = np.linspace(-0.05,0.05,100)

plt.figure()
for time in zeroed_ds['time'].values[0:1]:
    time_ds = zeroed_ds.sel(time=time, direction='up')
    hist = time_ds['current'].plot.hist(bins=bins)
    time_ds = zeroed_ds.sel(time=time, direction='down')
    hist = time_ds['current'].plot.hist(bins=bins)
    
# sweep_avg = weird_ds.mean('sweep')
# plt.plot(sweep_avg['time'].values, sweep_avg['voltage'].values, 'b', label='Average')
plt.show()

## Joe analysis

In [21]:
combined_all_ds

<xarray.Dataset>
Dimensions:                 (direction: 2, probe: 2, shot_number: 3, time: 5000, ts_radial_pos: 46)
Coordinates:
  * direction               (direction) object 'up' 'down'
  * probe                   (probe) object 'S' 'L'
  * shot_number             (shot_number) int64 398 400 411
    ts_number               (shot_number) float64 ...
    ts_timestamp            (shot_number) float64 ...
    ts_time                 (shot_number) datetime64[ns] ...
    adc_index               (shot_number) float64 ...
    adc_time                (shot_number) datetime64[ns] ...
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
Dimensions without coordinates: time
Data variables:
    voltage                 (shot_number, probe, direction, time) float64 dask.array<chunksize=(1, 2, 2, 5000), meta=np.ndarray>
    current                 (shot_number, probe, direction, time) float64 dask.array<chunksize=(1, 2, 2, 5000), meta=np.ndarray>
    shot_time           

In [18]:
fig, ax = plt.subplots(2)
((combined_all_ds.max('ts_radial_pos').sel(probe='S', direction='up').set_coords('voltage')['current'])).plot.line(x='voltage', hue='shot_number', ax=ax[0])
((combined_all_ds.max('ts_radial_pos').sel(probe='L', direction='up').set_coords('voltage')['current'])).plot.line(x='voltage', hue='shot_number', ax=ax[1])
ax[0].axhline(0, linestyle='--', color='grey', linewidth=1)
ax[1].axhline(0, linestyle='--', color='grey', linewidth=1)
plt.show()

In [26]:
# for Joe's analysis
anglescan_ds = combined_all_ds.assign_coords(tilt=helium_meta_ds['shot_tilt'].round()).mean('direction')
anglescan_ds

<xarray.Dataset>
Dimensions:                 (probe: 2, shot_number: 3, time: 5000, ts_radial_pos: 46)
Coordinates:
  * probe                   (probe) object 'S' 'L'
  * shot_number             (shot_number) int32 398 400 411
    ts_number               (shot_number) float64 ...
    ts_timestamp            (shot_number) float64 ...
    ts_time                 (shot_number) datetime64[ns] ...
    adc_index               (shot_number) float64 ...
    adc_time                (shot_number) datetime64[ns] ...
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
    tilt                    (shot_number) float64 8.0 8.0 5.0
Dimensions without coordinates: time
Data variables:
    voltage                 (shot_number, probe, time) float64 dask.array<chunksize=(1, 2, 5000), meta=np.ndarray>
    current                 (shot_number, probe, time) float64 dask.array<chunksize=(1, 2, 5000), meta=np.ndarray>
    shot_time               (shot_number, probe, time) float64 

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


(46,)


# Trying to find problem with time dimension
Found it by looking at teh individual time values on each dataset and noticing slight differences. Fixed by dropping the coordinate from time, which can be added back in later if necessary.

In [9]:
analysed_infos_df['shot_index'].loc[indices[0]]

143

In [22]:
ds_list = []
slicey = slice(0, 2)
for i, ds_file in enumerate(files_oi[slicey]):
    ds = xr.open_dataset(ds_file).sel(sweep=slice(0,699)).expand_dims({'shot_number':[analysed_infos_df['shot_index'].loc[indices[i]]]}).reset_index('time').mean('direction').mean('sweep')
    ds_list.append(ds)

shot_number_da = xr.DataArray(analysed_infos_df.loc[indices[slicey]]['shot_index'].values, dims=['shot_number'], name='shot_number')
autocombined_ds = xr.concat(ds_list, dim=shot_number_da)
print(autocombined_ds)

<xarray.Dataset>
Dimensions:      (probe: 2, shot_number: 2, time: 5000)
Coordinates:
  * probe        (probe) object 'S' 'L'
  * shot_number  (shot_number) int64 143 144
    time_        (shot_number, time) float64 0.0 1e-06 ... 0.004998 0.004999
Dimensions without coordinates: time
Data variables:
    voltage      (shot_number, probe, time) float64 88.51 88.59 ... -967.3
    current      (shot_number, probe, time) float64 0.2893 0.2893 ... -0.01502
    shot_time    (shot_number, probe, time) float64 4.496 4.496 ... 4.496 4.496
    start_time   (shot_number, probe) float64 4.494 4.494 4.494 4.494


In [24]:
for ds_file in files_oi[0:1]:
    ds = xr.open_dataset(ds_file)
    print(ds['time'].max())
    print(ds_file, ds['time'].values, ds.mean('direction').mean('sweep')['time'].values)

<xarray.DataArray 'time' ()>
array(0.004999)
analysed_1/a143_148_6698301520675232768.nc [0.0000000e+00 1.0000001e-06 2.0000002e-06 ... 4.9970005e-03 4.9980005e-03
 4.9990005e-03] [0.0000000e+00 1.0000001e-06 2.0000002e-06 ... 4.9970005e-03 4.9980005e-03
 4.9990005e-03]
